In [1]:
import numpy as np
import pandas as pd
import time
from scipy.optimize import minimize

In [3]:
## Read in csv file from IESO website, ***fill blank entries with 0*** (temp)
data_6months = pd.read_excel("SampleWeatherData.xlsx", index_col=0)
data_6months ['Month 1&2'] = data_6months ['Month 1&2'].apply(lambda x: (2.02887*x + 39.633)/100)
data_6months ['Month 3&4'] = data_6months ['Month 3&4'].apply(lambda x: (0.453122*x + 19.8276)/100)
data_6months ['Month 5&6'] = data_6months ['Month 5&6'].apply(lambda x: (1.13665*x - 11.0085)/100)
data_6months ['Month 7&8'] = data_6months ['Month 7&8'].apply(lambda x: (1.90245*x - 23.2826)/100)
data_6months ['Month 9&10'] = data_6months ['Month 9&10'].apply(lambda x: (1.39145*x - 8.97971)/100)
data_6months ['Month 11&12'] = data_6months ['Month 11&12'].apply(lambda x: (1.72767*x + 21.3536)/100)



In [4]:
load_data = pd.read_excel('load_data.xlsx', index_col=0)
load_data


,Month 1 & Month 2,Month 3 & Month 4,Month 5 & Month 6,Month 7 & Month 8,Month 9 & Month 10,Month 11 & Month 12
Hour,,,,,,
0,1438.395,604.9708,902.2919,1075.939,1075.725,1314.213
1,1414.594,605.7097,888.1929,1045.151,1034.786,1320.044
2,1418.216,605.0158,925.5618,1054.296,1052.425,1300.168
3,1432.950,604.2784,907.4609,1024.815,1026.229,1313.003
4,1407.696,610.8801,923.2916,1050.769,1032.733,1303.902
5,1399.250,606.2308,1013.8140,1169.434,1142.472,1320.345
6,1425.513,628.7807,1106.3390,1268.285,1221.761,1344.123
7,1487.829,634.4462,1158.4610,1334.432,1269.521,1403.989
8,1500.000,583.6489,1183.6180,1381.811,1270.018,1467.105


In [5]:
## Initialize all constants

ESB = 10000
PSB = 5000

# Revenue
KPDDR = 1200.264/8720       ## $856.436 /kW year

# Monthly Bill 
WMST = 0.003499             ## $/kWh 
KDC = 0.43

# Asset Costs
KPB = 40
PS = 5000 
KEB = 500
ES = 10000

# Cycle Constraint
KNC = 67

In [6]:
## x[0:24] = PCEA
## x[24:48] = ESB
## x[48:72] = PCDDR
start_time = time.time()
def constraint1(x):
    for i in range(24):
        if (i == 0):
            x[24] = 0
        else:
            x[24+i] = x[24+i-1] + x[i] - x[48+i-1]
    return x[0:24] + x[24:48] - x[48:72]
    
def constraint2(x):
    for i in range(24):
        if (i == 0):
            x[24] = 0
        else:
            x[24+i] = x[24+i-1] + x[i] - x[48+i-1]
    return ESB - (x[0:24]+ x[24:48] - x[48:72])

def constraint4(x): 
    return 5000 - (abs(x[0:24]) + abs(x[48:72]))

def constraint3(x):
    NC = 0 
    for i in range(24):
        if (i == 0): 
            NC = NC + 0
        else:
            if ((x[24+i] - x[24+i-1]) >= 0):
                NC = NC + (x[24+i] - x[24+i-1])
            else: 
                NC = NC + 0
                
    return KNC - NC/ESB*30

power = ((-PSB, PSB),) * 24
storage = ((0, ESB),) * 24
DDR = ((0,PSB),) * 24

##Objective function
def MEC(x):           #      KPDDR*PCDDR - (PCEA+PDLL)*(HOEP + WMST) - PDF*(GAC+CBR) - KDC*max(PDLL+PCEA)
    return -(
        #Revenue
        KPDDR*sum(30*(sum(np.array([x[48:72]])))) - 
        #Costs
            #MEC
             sum(30*sum((load_data.iloc[:,month].to_numpy() + np.array([x[0:24]])) * 
                        (data_6months.iloc[:,month].to_numpy() + WMST))) - 
            #MDC
            #30*KDC*(PDLL + PCEA)
            30*KDC*np.amax(load_data.iloc[:,month].to_numpy() + np.array([x[0:24]]))
            )


x0 = np.array([np.ones(24), np.ones(24), np.ones(24)])
bounds = (power + storage + DDR)
cons1 = {'type': 'ineq', 'fun': constraint1}
cons2 = {'type': 'ineq', 'fun': constraint2}
cons3 = {'type': 'ineq', 'fun': constraint3}
cons4 = {'type': 'ineq', 'fun': constraint4}
cons = ([cons1, cons2, cons3,cons4])

pcea_solutions = []
esb_solutions = []
ddr_solutions = []

mec = 0 

for month in range(6):
    sol = minimize(MEC, x0, method='SLSQP',
                     bounds=bounds,
                     constraints=cons,
                     options= {'maxiter':150,'disp':True})
    mec = mec + sol.fun
    pcea_solutions.append(sol.x[0:24])
    esb_solutions.append(sol.x[24:48])
    ddr_solutions.append(sol.x[48:72])
      

print(-2*mec)
print("--- %s seconds ---" % (time.time() - start_time))

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 358873.48647829494
            Iterations: 126
            Function evaluations: 9587
            Gradient evaluations: 126
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 96963.13171037931
            Iterations: 29
            Function evaluations: 2257
            Gradient evaluations: 29
Iteration limit exceeded    (Exit mode 9)
            Current function value: 10723.99418035677
            Iterations: 151
            Function evaluations: 11229
            Gradient evaluations: 151
Iteration limit exceeded    (Exit mode 9)
            Current function value: 192107.55400800635
            Iterations: 151
            Function evaluations: 11269
            Gradient evaluations: 150
Iteration limit exceeded    (Exit mode 9)
            Current function value: 74379.42337845525
            Iterations: 151
            Function evaluations: 11256
    

In [ ]:
df_sol = pd.concat([pd.DataFrame(data=pcea_solutions).transpose(), 
                    pd.DataFrame(data=esb_solutions).transpose(), 
                    pd.DataFrame(data=ddr_solutions).transpose()], axis=1)
df_sol